## Addresses

Create an `Address` object from a _bech32-encoded_ string:

In [122]:
from multiversx_sdk_core import Address

address = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")

print("Address (bech32-encoded)", address.bech32())
print("Public key (hex-encoded):", address.hex())
print("Public key (hex-encoded):", address.pubkey.hex())


Address (bech32-encoded) erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th
Public key (hex-encoded): 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1
Public key (hex-encoded): 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


... or from a _hex-encoded_ string - note that you have to provide the address prefix, also known as the **HRP** (_human-readable part_ of the address):

In [123]:
address = Address.from_hex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1", "erd")


... or from a raw public key:

In [124]:
pubkey = bytes.fromhex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1")
address = Address(pubkey, "erd")


Alternatively, you can use an `AddressFactory` (initialized with a specific **HRP**) to create addresses:

In [125]:
from multiversx_sdk_core import AddressFactory

factory = AddressFactory("erd")

address = factory.create_from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
address = factory.create_from_hex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1")
address = factory.create_from_pubkey(bytes.fromhex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1"))


Addresses can be converted from one representation to another as follows:

In [126]:
print(address.bech32())
print(address.hex())


erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th
0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


... or using a converter:

In [127]:
from multiversx_sdk_core import AddressConverter

converter = AddressConverter("erd")

pubkey = converter.bech32_to_pubkey("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
bech32 = converter.pubkey_to_bech32(bytes.fromhex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1"))


Getting the shard of an address:

In [128]:
print("Shard:", address.get_shard())


Shard: 1


Checking whether an address is a smart contract:

In [129]:
address = Address.from_bech32("erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm")

print("Is contract:", address.is_smart_contract())


Is contract: True


## EGLD / ESDT transfers

Create an EGLD transfer:

In [130]:
from multiversx_sdk_core import Address, TokenPayment, Transaction

tx = Transaction(
    nonce=90,
    sender=Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th"),
    receiver=Address.from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx"),
    value=TokenPayment.egld_from_amount("1.0"),
    gas_limit=50000,
    gas_price=1000000000,
    chain_id="D",
    version=1
)

print(tx.to_dictionary())


OrderedDict([('nonce', 90), ('value', '1000000000000000000'), ('receiver', 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 50000), ('chainID', 'D'), ('version', 1), ('signature', '')])


In case you are using a **guarded** account you should also provide the _guardian_ argument when creating the transaction.

We'll see later how to [sign](#signing-objects) and [broadcast](#broadcasting-transactions) a transaction.

Create an EGLD transfer, but this time with a payload (data):

In [131]:
from multiversx_sdk_core import TransactionPayload

data = TransactionPayload.from_str("for the book")

tx = Transaction(
    nonce=91,
    sender=Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th"),
    receiver=Address.from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx"),
    value=TokenPayment.egld_from_amount("3.0"),
    data=data,
    gas_limit=50000 + 1500 * data.length(),
    gas_price=1000000000,
    chain_id="D",
    version=1
)

print(tx.to_dictionary())


OrderedDict([('nonce', 91), ('value', '3000000000000000000'), ('receiver', 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 68000), ('data', 'Zm9yIHRoZSBib29r'), ('chainID', 'D'), ('version', 1), ('signature', '')])


Alternatively, we can create an EGLD transfer using a **transaction builder** (as we will see below, transaction builders are more commonly used). But before that, we have to create a configuration object (for any builder that we might use):

In [132]:
from multiversx_sdk_core.transaction_builders import DefaultTransactionBuildersConfiguration

config = DefaultTransactionBuildersConfiguration(chain_id="D")


The **transaction builder** is parametrized at instantiation, and the transaction is obtained by invoking the `build()` method:

In [133]:
from multiversx_sdk_core.transaction_builders import EGLDTransferBuilder

alice = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
bob = Address.from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx")
payment = TokenPayment.egld_from_amount("1.00")

# With "data" field
builder = EGLDTransferBuilder(
    config=config,
    sender=alice,
    receiver=bob,
    payment=payment,
    data="for the book",
    # Nonce can be set now, or later, prior signing
    nonce=42
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)


Transaction: OrderedDict([('nonce', 42), ('value', '1000000000000000000'), ('receiver', 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 68000), ('data', 'Zm9yIHRoZSBib29r'), ('chainID', 'D'), ('version', 2), ('signature', '')])
Transaction data: for the book


Create a single ESDT transfer:

In [134]:
from multiversx_sdk_core.transaction_builders import ESDTTransferBuilder

payment = TokenPayment.fungible_from_amount("TEST-8b028f", "100.00", 2)

builder = ESDTTransferBuilder(
    config=config,
    sender=alice,
    receiver=bob,
    payment=payment
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)


Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 410000), ('data', 'RVNEVFRyYW5zZmVyQDU0NDU1MzU0MmQzODYyMzAzMjM4NjZAMjcxMA=='), ('chainID', 'D'), ('version', 2), ('signature', '')])
Transaction data: ESDTTransfer@544553542d386230323866@2710


Create a single NFT transfer:

In [135]:
from multiversx_sdk_core.transaction_builders import ESDTNFTTransferBuilder

payment = TokenPayment.non_fungible("TEST-38f249", 1)

builder = ESDTNFTTransferBuilder(
    config=config,
    sender=alice,
    destination=bob,
    payment=payment
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)


Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 1213500), ('data', 'RVNEVE5GVFRyYW5zZmVyQDU0NDU1MzU0MmQzMzM4NjYzMjM0MzlAMDFAMDFAODA0OWQ2MzllNWE2OTgwZDFjZDIzOTJhYmNjZTQxMDI5Y2RhNzRhMTU2MzUyM2EyMDJmMDk2NDFjYzI2MThmOA=='), ('chainID', 'D'), ('version', 2), ('signature', '')])
Transaction data: ESDTNFTTransfer@544553542d333866323439@01@01@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8


Create a single SFT transfer (almost the same as above, the only difference being that for the token payment object we additionally use a quantity, as an integer):

In [136]:
payment = TokenPayment.semi_fungible("SEMI-9efd0f", 1, 5)

builder = ESDTNFTTransferBuilder(
    config=config,
    sender=alice,
    destination=bob,
    payment=payment
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)


Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 1213500), ('data', 'RVNEVE5GVFRyYW5zZmVyQDUzNDU0ZDQ5MmQzOTY1NjY2NDMwNjZAMDFAMDVAODA0OWQ2MzllNWE2OTgwZDFjZDIzOTJhYmNjZTQxMDI5Y2RhNzRhMTU2MzUyM2EyMDJmMDk2NDFjYzI2MThmOA=='), ('chainID', 'D'), ('version', 2), ('signature', '')])
Transaction data: ESDTNFTTransfer@53454d492d396566643066@01@05@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8


Create a multiple ESDT / NFT transfer:

In [137]:
from multiversx_sdk_core.transaction_builders import MultiESDTNFTTransferBuilder

payment_one = TokenPayment.non_fungible("TEST-38f249", 1)
payment_two = TokenPayment.fungible_from_amount("BAR-c80d29", "10.00", 18)

builder = MultiESDTNFTTransferBuilder(
    config=config,
    sender=alice,
    destination=bob,
    payments=[payment_one, payment_two]
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)


Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 2284000), ('data', 'TXVsdGlFU0RUTkZUVHJhbnNmZXJAODA0OWQ2MzllNWE2OTgwZDFjZDIzOTJhYmNjZTQxMDI5Y2RhNzRhMTU2MzUyM2EyMDJmMDk2NDFjYzI2MThmOEAwMkA1NDQ1NTM1NDJkMzMzODY2MzIzNDM5QDAxQDAxQDQyNDE1MjJkNjMzODMwNjQzMjM5QEA4YWM3MjMwNDg5ZTgwMDAw'), ('chainID', 'D'), ('version', 2), ('signature', '')])
Transaction data: MultiESDTNFTTransfer@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8@02@544553542d333866323439@01@01@4241522d633830643239@@8ac7230489e80000


## Relayed Transactions

First, we get the newtwork configuration using the network providers.

In [138]:
from multiversx_sdk_network_providers import ProxyNetworkProvider

provider = ProxyNetworkProvider("https://devnet-gateway.multiversx.com")
network_config = provider.get_network_config()


### Relayed V1

In [139]:
from pathlib import Path
from multiversx_sdk_core.transaction_builders.relayed_v1_builder import RelayedTransactionV1Builder
from multiversx_sdk_core import Transaction, Address, TransactionPayload
from multiversx_sdk_wallet.user_signer import UserSigner


signer = UserSigner.from_pem_file(Path("./testwallets/bob.pem"))

inner_tx = Transaction(
    chain_id=network_config.chain_id,
    sender=Address.from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx"),
    receiver=Address.from_bech32("erd1qqqqqqqqqqqqqpgqqczn0ccd2gh8eqdswln7w9vzctv0dwq7d8ssm4y34z"),
    gas_limit=60000000,
    nonce=198,
    data=TransactionPayload.from_str("add@05")
)
inner_tx.signature = signer.sign(inner_tx.serialize_for_signing())

relayed_builder = RelayedTransactionV1Builder()
relayed_builder.set_inner_transaction(inner_tx)
relayed_builder.set_relayer_nonce(2627)
relayed_builder.set_network_config(network_config)
relayed_builder.set_relayer_address(Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th"))

relayed_tx = relayed_builder.build()
print(relayed_tx.to_dictionary())


OrderedDict([('nonce', 2627), ('value', '0'), ('receiver', 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 61052000), ('data', 'cmVsYXllZFR4QDdiMjI2ZTZmNmU2MzY1MjIzYTMxMzkzODJjMjI3MzY1NmU2NDY1NzIyMjNhMjI2NzQ1NmU1NzRmNjU1NzZkNmQ0MTMwNjMzMDZhNmI3MTc2NGQzNTQyNDE3MDdhNjE2NDRiNDY1NzRlNTM0ZjY5NDE3NjQzNTc1MTYzNzc2ZDQ3NTA2NzNkMjIyYzIyNzI2NTYzNjU2OTc2NjU3MjIyM2EyMjQxNDE0MTQxNDE0MTQxNDE0MTQxNDE0NjQxNDE1OTQ2NGUyYjRkNGU1NTY5MzUzODY3NjI0MjMzMzUyYjYzNTY2NzczNGM1OTM5NzI2NzY1NjE2NTQ1M2QyMjJjMjI3NjYxNmM3NTY1MjIzYTMwMmMyMjY3NjE3MzUwNzI2OTYzNjUyMjNhMzEzMDMwMzAzMDMwMzAzMDMwMzAyYzIyNjc2MTczNGM2OTZkNjk3NDIyM2EzNjMwMzAzMDMwMzAzMDMwMmMyMjY0NjE3NDYxMjIzYTIyNTk1NzUyNmI1MTQ0NDEzMTIyMmMyMjczNjk2NzZlNjE3NDc1NzI2NTIyM2EyMjQ3NGM0MzM0NGE1OTM1NGMzMDc1NmY0YTRkNzA0ODc1MzQ0ODUwNTc2NzQ3NDE0ODQ3NmY0NzZhN2E0NzYxMzYzNTJiNzU0ODMwMzAyZjc3NGM1MTU5NDU1MTUyMzA2NDVhNGM2MzUxMzQ1NDZkNzE1NTM5NGI2ZDUx

### Relayed V2

In [140]:
from multiversx_sdk_core.transaction_builders.relayed_v2_builder import RelayedTransactionV2Builder
from multiversx_sdk_core import Transaction, Address, TransactionPayload
from multiversx_sdk_wallet.user_signer import UserSigner


signer = UserSigner.from_pem_file(Path("./testwallets/bob.pem"))

inner_tx = Transaction(
            chain_id=network_config.chain_id,
            sender=Address.from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx"),
            receiver=Address.from_bech32("erd1qqqqqqqqqqqqqpgqqczn0ccd2gh8eqdswln7w9vzctv0dwq7d8ssm4y34z"),
            gas_limit=0,
            nonce=15,
            data=TransactionPayload.from_str("add@05")
        )
inner_tx.signature = signer.sign(inner_tx.serialize_for_signing())

builder = RelayedTransactionV2Builder()
builder.set_inner_transaction(inner_tx)
builder.set_inner_transaction_gas_limit(60_000_000)
builder.set_relayer_nonce(37)
builder.set_network_config(network_config)
builder.set_relayer_address(Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th"))

relayed_tx = builder.build()
print(relayed_tx.to_dictionary())


OrderedDict([('nonce', 37), ('value', '0'), ('receiver', 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 60381500), ('data', 'cmVsYXllZFR4VjJAMDAwMDAwMDAwMDAwMDAwMDA1MDAwNjA1MzdlMzBkNTIyZTdjODFiMDc3ZTdlNzE1ODJjMmQ4ZjZiODFlNjllMUAwZkA2MTY0NjQ0MDMwMzVANjUwNzJhOTExOGRiMTQ3NDdhNGYxY2Y4NTdhNmU0MzczZTNlOWVhZDZlYTYwMzZmNTExMWJmNjg4MDNjNWNhZGZkYzQ0ZDM3YTIxYjkyMDQ0OWI1ZTgzYzc4Mzk3NTQyNWMzZjQ3NjAyMDE3M2FjMGZjNzE0ODE2NDZkNGE5MDA='), ('chainID', 'D'), ('version', 2), ('signature', '')])


## Contract deployments and interactions

Create a transaction to deploy a smart contract:

In [141]:
from pathlib import Path

from multiversx_sdk_core import CodeMetadata
from multiversx_sdk_core.transaction_builders import ContractDeploymentBuilder

metadata = CodeMetadata(upgradeable=True, readable=True, payable=True, payable_by_contract=True)

builder = ContractDeploymentBuilder(
    config,
    owner=alice,
    deploy_arguments=[42, "test"],
    code_metadata=metadata,
    code=Path("./contracts/counter.wasm").read_bytes(),
    gas_limit=10000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)


Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq6gq4hu'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 10000000), ('data', 'MDA2MTczNmQwMTAwMDAwMDAxMTUwNDYwMDM3ZjdmN2UwMTdmNjAwMjdmN2YwMTdlNjAwMTdlMDA2MDAwMDAwMjQyMDMwMzY1NmU3NjExNjk2ZTc0MzYzNDczNzQ2ZjcyNjE2NzY1NTM3NDZmNzI2NTAwMDAwMzY1NmU3NjEwNjk2ZTc0MzYzNDczNzQ2ZjcyNjE2NzY1NGM2ZjYxNjQwMDAxMDM2NTZlNzYwYjY5NmU3NDM2MzQ2NjY5NmU2OTczNjgwMDAyMDMwNTA0MDMwMzAzMDMwNDA1MDE3MDAxMDEwMTA1MDMwMTAwMDIwNjA4MDE3ZjAxNDE5MDg4MDQwYjA3MmYwNTA2NmQ2NTZkNmY3Mjc5MDIwMDA0Njk2ZTY5NzQwMDAzMDk2OTZlNjM3MjY1NmQ2NTZlNzQwMDA0MDk2NDY1NjM3MjY1NmQ2NTZlNzQwMDA1MDM2NzY1NzQwMDA2MGE4YTAxMDQxMzAwNDE4MDg4ODA4MDAwNDEwNzQyMDExMDgwODA4MDgwMDAxYTBiMmUwMTAxN2U0MTgwODg4MDgwMDA0MTA3NDE4MDg4ODA4MDAwNDEwNzEwODE4MDgwODAwMDQyMDE3YzIyMDAxMDgwODA4MDgwMDAxYTIwMDAxMDgyODA4MDgwMDAwYjJlMDEwMTdlNDE4MDg4ODA4MDAwNDEwNzQxODA4ODgwODAwMDQxMDcxMDgxODA4MD

Create a transaction to upgrade an existing smart contract:

In [142]:
from multiversx_sdk_core.transaction_builders import ContractUpgradeBuilder

contract_address = Address.from_bech32("erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm")
owner = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
metadata = CodeMetadata(upgradeable=True, readable=True, payable=True, payable_by_contract=True)

builder = ContractUpgradeBuilder(
    config,
    contract=contract_address,
    owner=owner,
    upgrade_arguments=[42, "test"],
    code_metadata=metadata,
    code=Path("./contracts/counter.wasm").read_bytes(),
    gas_limit=10000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)


Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 10000000), ('data', 'dXBncmFkZUNvbnRyYWN0QDAwNjE3MzZkMDEwMDAwMDAwMTE1MDQ2MDAzN2Y3ZjdlMDE3ZjYwMDI3ZjdmMDE3ZTYwMDE3ZTAwNjAwMDAwMDI0MjAzMDM2NTZlNzYxMTY5NmU3NDM2MzQ3Mzc0NmY3MjYxNjc2NTUzNzQ2ZjcyNjUwMDAwMDM2NTZlNzYxMDY5NmU3NDM2MzQ3Mzc0NmY3MjYxNjc2NTRjNmY2MTY0MDAwMTAzNjU2ZTc2MGI2OTZlNzQzNjM0NjY2OTZlNjk3MzY4MDAwMjAzMDUwNDAzMDMwMzAzMDQwNTAxNzAwMTAxMDEwNTAzMDEwMDAyMDYwODAxN2YwMTQxOTA4ODA0MGIwNzJmMDUwNjZkNjU2ZDZmNzI3OTAyMDAwNDY5NmU2OTc0MDAwMzA5Njk2ZTYzNzI2NTZkNjU2ZTc0MDAwNDA5NjQ2NTYzNzI2NTZkNjU2ZTc0MDAwNTAzNjc2NTc0MDAwNjBhOGEwMTA0MTMwMDQxODA4ODgwODAwMDQxMDc0MjAxMTA4MDgwODA4MDAwMWEwYjJlMDEwMTdlNDE4MDg4ODA4MDAwNDEwNzQxODA4ODgwODAwMDQxMDcxMDgxODA4MDgwMDA0MjAxN2MyMjAwMTA4MDgwODA4MDAwMWEyMDAwMTA4MjgwODA4MDAwMGIyZTAxMDE3ZTQxODA4ODgwODAwMDQxMDc0MTgwODg4MD

Create a transaction that invokes a smart contract function:

In [143]:
from multiversx_sdk_core.transaction_builders import ContractCallBuilder

contract_address = Address.from_bech32("erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm")

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="foo",
    caller=alice,
    call_arguments=[42, "test", bob],
    gas_limit=10000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)


Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 10000000), ('data', 'Zm9vQDJhQDc0NjU3Mzc0QDgwNDlkNjM5ZTVhNjk4MGQxY2QyMzkyYWJjY2U0MTAyOWNkYTc0YTE1NjM1MjNhMjAyZjA5NjQxY2MyNjE4Zjg='), ('chainID', 'D'), ('version', 2), ('signature', '')])
Transaction data: foo@2a@74657374@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8


Now, let's create a call that also transfers one or more tokens (**transfer & execute**):

In [144]:
transfers = [
    TokenPayment.non_fungible("TEST-38f249", 1),
    TokenPayment.fungible_from_amount("BAR-c80d29", "10.00", 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="hello",
    caller=alice,
    call_arguments=[42, "test", bob],
    gas_limit=10000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)


Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 10000000), ('data', 'TXVsdGlFU0RUTkZUVHJhbnNmZXJAMDAwMDAwMDAwMDAwMDAwMDA1MDBlMGI3N2YxZWRmYjAxY2I3ODZhMzkxMjBmMDJjMzFmZjVmZTQwYWFkODk3NEAwMkA1NDQ1NTM1NDJkMzMzODY2MzIzNDM5QDAxQDAxQDQyNDE1MjJkNjMzODMwNjQzMjM5QEA4YWM3MjMwNDg5ZTgwMDAwQDY4NjU2YzZjNmZAMmFANzQ2NTczNzRAODA0OWQ2MzllNWE2OTgwZDFjZDIzOTJhYmNjZTQxMDI5Y2RhNzRhMTU2MzUyM2EyMDJmMDk2NDFjYzI2MThmOA=='), ('chainID', 'D'), ('version', 2), ('signature', '')])
Transaction data: MultiESDTNFTTransfer@00000000000000000500e0b77f1edfb01cb786a39120f02c31ff5fe40aad8974@02@544553542d333866323439@01@01@4241522d633830643239@@8ac7230489e80000@68656c6c6f@2a@74657374@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8


## Contract queries

In order to create a contract query and run it against a network provider (more details about **network providers** can be found below), do as follows:

In [145]:
from multiversx_sdk_core import ContractQueryBuilder
from multiversx_sdk_core.interfaces import IAddress
from multiversx_sdk_network_providers import ApiNetworkProvider

contract: IAddress = Address.from_bech32("erd1qqqqqqqqqqqqqpgqnzm7yhayarylux045qlm4lgzmtcsgrqg396qr9kupx")

builder = ContractQueryBuilder(
    contract=contract,
    function="getSum",
    call_arguments=[],
    caller=alice
)

query = builder.build()

network_provider = ApiNetworkProvider("https://devnet-api.multiversx.com")
response = network_provider.query_contract(query)

print("Return code:", response.return_code)
print("Return data:", response.return_data)


Return code: ok
Return data: ['LQ==']


## Creating wallets

Mnemonic generation is based on [`trezor/python-mnemonic`](https://github.com/trezor/python-mnemonic) and can be achieved as follows:

In [146]:
from multiversx_sdk_wallet import Mnemonic

mnemonic = Mnemonic.generate()
words = mnemonic.get_words()

print(words)


['shallow', 'seed', 'final', 'slide', 'south', 'kiwi', 'all', 'used', 'mom', 'song', 'transfer', 'vital', 'stuff', 'divide', 'laundry', 'caught', 'aisle', 'reflect', 'fuel', 'large', 'walk', 'elite', 'alarm', 'sock']


The mnemonic can be saved to a keystore file:

In [147]:
from multiversx_sdk_wallet import UserWallet

wallet = UserWallet.from_mnemonic(mnemonic.get_text(), "password")
wallet.save(Path("./output/walletWithMnemonic.json"))


Given a mnemonic, one can derive keypairs:

In [148]:
secret_key = mnemonic.derive_key(0)
public_key = secret_key.generate_public_key()

print("Secret key", secret_key.hex())
print("Public key", public_key.hex())


Secret key 7d966af37adfb4afdfdd3f57b840e22e5ea012a265a4a0eaaf23aa286648b606
Public key d66b160e7c0626b723e64a4d3c7f82cade014b5b4b7f28eead2ac755840fa1ac


A keypair can be saved as a JSON wallet:

In [149]:
wallet = UserWallet.from_secret_key(secret_key, "password")
wallet.save(Path("./output/wallet.json"), address_hrp="erd")


... or as a PEM wallet (usually not recommended):

In [150]:
from multiversx_sdk_wallet import UserPEM

label = Address(public_key.buffer, "erd").bech32()
pem = UserPEM(label=label, secret_key=secret_key)
pem.save(Path("./output/wallet.pem"))


## Loading wallets

This is not a very common use-case - you might refer to [signing objects](#signing-objects) instead.

Load a keystore that holds an **encrypted mnemonic** (and perform wallet derivation at the same time):

In [151]:
from multiversx_sdk_wallet import UserWallet

secret_key = UserWallet.load_secret_key(Path("./testwallets/withMnemonic.json"), "password", address_index=0)
address = secret_key.generate_public_key().to_address("erd")

print("Secret key", secret_key.hex())
print("Address", address)


Secret key 413f42575f7f26fad3317a778771212fdb80245850981e48b58a4f25e344e8f9
Address erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th


Load a keystore that holds an **encrypted secret** key:

In [152]:
secret_key = UserWallet.load_secret_key(Path("./testwallets/alice.json"), "password")
address = secret_key.generate_public_key().to_address("erd")

print("Secret key", secret_key.hex())
print("Address", address)


Secret key 413f42575f7f26fad3317a778771212fdb80245850981e48b58a4f25e344e8f9
Address erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th


Load the secret key from a PEM file:

In [153]:
from multiversx_sdk_wallet import UserPEM

pem = UserPEM.from_file(Path("./testwallets/alice.pem"))

print("Secret key", pem.secret_key.hex())
print("Public key", pem.public_key.hex())


Secret key 413f42575f7f26fad3317a778771212fdb80245850981e48b58a4f25e344e8f9
Public key 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


## Signing objects

Creating a `UserSigner` from a JSON wallet:

In [154]:
from multiversx_sdk_wallet import UserSigner

signer = UserSigner.from_wallet(Path("./testwallets/alice.json"), "password")


Creating a `UserSigner` from a PEM file:

In [155]:
signer = UserSigner.from_pem_file(Path("./testwallets/alice.pem"))


Signable objects (messages, transactions) must adhere to the following interface:
```
class ISignable(Protocol):
    def serialize_for_signing(self) -> bytes: ...
```

Both `Transaction` and `Message` - defined in `multiversx_sdk_core` - implement `ISignable`.

Signing a transaction:

In [156]:
from multiversx_sdk_core import Transaction

tx = Transaction(
    nonce=90,
    sender=Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th"),
    receiver=Address.from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx"),
    value=TokenPayment.egld_from_amount("1.0"),
    gas_limit=50000,
    gas_price=1000000000,
    chain_id="D",
    version=1
)

tx.signature = signer.sign(tx.serialize_for_signing())
print("Signature", tx.signature.hex())


Signature 762e224eb5b8f2e801c2f0ea5ca1fb0aa08d4566a04f3846b250e222b76cd5d2141f5f131afecda9a3c5433f41790f33c45a559bea9a7d28ba964308fbcbfd0e


Signing an arbitrary message:

In [157]:
from multiversx_sdk_core import MessageV1

message = MessageV1.from_string("hello")
message.signature = signer.sign(message.data)

print("Signature", message.signature.hex())


Signature 66dd503199222d3104cb5381da953b14c895ca564579f98934c4e54a45d75f097da2a0c30060c0e7d014928b8e54509335cadc69b2bb1940cd59bd0c1bb80b0b


## Verifying signatures

Creating a `UserVerifier`:

In [158]:
from multiversx_sdk_core import Address
from multiversx_sdk_wallet import UserVerifier

alice = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
bob = Address.from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx")
alice_verifier = UserVerifier.from_address(alice)
bob_verifier = UserVerifier.from_address(bob)


For objects to support signature verification, they must adhere to the following interface:

```
class IVerifiable(Protocol):
    signature: ISignature
    def serialize_for_signing(self) -> bytes: ...
```

Both `Transaction` and `Message` - defined in `multiversx_sdk_core` - implement `IVerifiable`.

Verifying a signature:

In [171]:
print(f"Is signature of Alice?", alice_verifier.verify(tx.serialize_for_signing(), tx.signature))
print(f"Is signature of Alice?", alice_verifier.verify(message.data, message.signature))
print(f"Is signature of Bob?", bob_verifier.verify(tx.serialize_for_signing(), tx.signature))
print(f"Is signature of Bob?", bob_verifier.verify(message.data, message.signature))


Is signature of Alice? True
Is signature of Alice? True
Is signature of Bob? False
Is signature of Bob? False


## Creating network providers

It's recommended to use the `multiversx_sdk_network_providers` components **as a starting point**. As your application matures, switch to using your own network provider (e.g. deriving from the default ones), tailored to your requirements.

Creating an API provider:

In [160]:
from multiversx_sdk_network_providers import ApiNetworkProvider

provider = ApiNetworkProvider("https://devnet-api.multiversx.com")


Creating a Proxy provider:

In [161]:
from multiversx_sdk_network_providers import ProxyNetworkProvider

provider = ProxyNetworkProvider("https://devnet-gateway.multiversx.com")


## Fetching network parameters

In order to fetch network parameters, do as follows:

In [162]:
config = provider.get_network_config()

print("Chain ID", config.chain_id)
print("Min gas price:", config.min_gas_price)


Chain ID D
Min gas price: 1000000000


## Fetching account state

The following snippet fetches (from the Network) the **nonce** and the **balance** of an account:

In [163]:
account_on_network = provider.get_account(alice)

print("Nonce", account_on_network.nonce)
print("Balance", account_on_network.balance)


Nonce 17301
Balance 15661807888855972487


When sending a number of transactions, you usually have to first fetch the account nonce from the network (see above), then manage it locally (e.g. increment upon signing & broadcasting a transaction):

In [164]:
from multiversx_sdk_core import AccountNonceHolder

nonce_holder = AccountNonceHolder(account_on_network.nonce)

tx.nonce = nonce_holder.get_nonce_then_increment()
# Then, sign & broadcast the transaction(s).


For further reference, please see [nonce management](https://docs.multiversx.com/integrators/creating-transactions/#nonce-management).

## Broadcasting transactions

Broadcast a single transaction:

In [165]:
alice = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")

tx = Transaction(
    sender=alice,
    receiver=alice,
    gas_limit=50000,
    chain_id="D"
)

alice_on_network = provider.get_account(alice)

tx.nonce = alice_on_network.nonce
tx.signature = signer.sign(tx.serialize_for_signing())

hash = provider.send_transaction(tx)
print("Transaction hash:", hash)


Transaction hash: 4a3a1cfcc1fe1bebe1de780d506c6e514c9b8e966632d9e621c187f50a660a8c


Broadcast multiple transactions:

In [166]:
tx_1 = Transaction(
    sender=alice,
    receiver=alice,
    gas_limit=50000,
    chain_id="D"
)

tx_2 = Transaction(
    sender=alice,
    receiver=alice,
    gas_limit=50000,
    chain_id="D"
)

tx_3 = Transaction(
    sender=alice,
    receiver=alice,
    gas_limit=50000,
    chain_id="D"
)

alice_on_network = provider.get_account(alice)
nonce_holder = AccountNonceHolder(account_on_network.nonce)

tx_1.nonce = nonce_holder.get_nonce_then_increment()
tx_2.nonce = nonce_holder.get_nonce_then_increment()
tx_3.nonce = nonce_holder.get_nonce_then_increment()

tx_1.signature = signer.sign(tx_1.serialize_for_signing())
tx_2.signature = signer.sign(tx_2.serialize_for_signing())
tx_3.signature = signer.sign(tx_3.serialize_for_signing())

hashes = provider.send_transactions([tx_1, tx_2, tx_3])
print("Transactions hashes:", hashes)


Transactions hashes: (3, {'0': '4a3a1cfcc1fe1bebe1de780d506c6e514c9b8e966632d9e621c187f50a660a8c', '1': 'a7a19cea88604b8e5f2a4742589ebae1e006fa0026fd76d036649b0f72508a9c', '2': '686e3d82dbbaee8f04ae4d82431d65fb0472befbae754a74be1ecdafc1e6a8d1'})


Now let's fetch a previously-broadcasted transaction:

In [167]:
tx_on_network = provider.get_transaction("09e3b68d39f3759913239b927c7feb9ac871c8877e76bc56e1be45a2a597eb53", with_process_status=True)
print("Status:", tx_on_network.status)
print("Is completed:", tx_on_network.is_completed)


Status: success
Is completed: True
